# State Vector and Orbital Elements

In [7]:
# Import the modules
import urllib
import warnings
from pathlib import Path

import numpy as np
import spiceypy as sp
from astropy.table import QTable
from astropy.time import Time
from astropy.utils.exceptions import AstropyWarning
from astroquery.jplhorizons import Horizons
from astroquery.jplsbdb import SBDB

warnings.filterwarnings('ignore', append=True, category=AstropyWarning)

def download_kernel(dl_path, dl_url):
    """Download kernel from the NASA NAIF repository

    Parameters
    ----------
    dl_path : str
        Download path on the local machine, relative to this function.
    dl_url : str
        Download url of the requested kernel file.
    """

    # Make directory if it does not exist
    dl_path = Path(dl_path)
    dl_path.mkdir(exist_ok=True)
    # Get the file name from the url
    file_name = dl_url.split('/')[-1]

    if not (dl_path/file_name).exists():
        # Download the file with the urllib  package
        urllib.request.urlretrieve(dl_url, dl_path/file_name)


# Load the SPICE kernels via a meta file
sp.furnsh('kernel_meta.txt')

NOW_UTC = Time.now().utc
NOW_ET = sp.utc2et(NOW_UTC.strftime('%Y-%m-%dT%H:%M:%S'))

# Get the G*M value for the Sun
_, GM_SUN = sp.bodvcd(bodyid=10, item='GM', maxn=1)
GM_SUN = GM_SUN[0]

## CODES 300 Asteroids bsp
A not-very-accurate 300 asteroid bsp made in 2010 ([README](https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/asteroids/AAREADME_Asteroids_SPKs.txt)).

> We have left here a single SPK containing orbits for a set of 300 asteroids, the orbits for which were produced by Jim Baer in 2010 using his own CODES software. Some of these data may also be not very accurate. But the convenience of having data for 300 objects in a single SPK file could be useful for some purposes where high accuracy is not important. Read the *.cmt" file for details about this collection.

> Usage
>
> This file should be used together with the DE405 planetary ephemeris since the integration was done using the DE405 ephemeris and with the frames kernel defining the ``ECLIPJ2000_DE405'' frame.
>
> ...
> 
> This frames kernel defines the Ecliptic and Equinox of the J2000 frame using the DE405 obliquity angle value (84381.412 arcseconds).

Also see the detailed [explanation by Thomas](https://towardsdatascience.com/space-science-with-python-quite-around-the-sun-6faa206a1210).



In [2]:
# Download the asteroids spk kernel file.
download_kernel(
    Path('../_kernels/spk/'),
    'https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/asteroids/codes_300ast_20100725.bsp'
)

# Download an auxiliary file from the repository that contains the NAIF ID
# codes and a reference frame kernel that is needed. Since we have a mixture
# of different kernel types we store the data in a sub-directory called _misc
download_kernel(
    Path('../_kernels/_misc/'),
    "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/asteroids/codes_300ast_20100725.tf"
)

Considering Thomas's explanation, let's check if their ``ECLIPJ2000_DE405`` == ``ECLIPJ2000``.

Note ``sxform`` gives 6x6 (for position & velocity) while ``pxform`` gives 3x3 (position only).

In [3]:
R_DE405_J2000_6x6 = sp.sxform('ECLIPJ2000_DE405', 'ECLIPJ2000', et=NOW_ET)
R_DE405_J2000_3x3 = sp.pxform('ECLIPJ2000_DE405', 'ECLIPJ2000', et=NOW_ET)

print('Transformation matrix between ECLIPJ2000_DE405 and ECLIPJ2000')
print(R_DE405_J2000_6x6.round(3))
print()
print(R_DE405_J2000_3x3.round(3))

Transformation matrix between ECLIPJ2000_DE405 and ECLIPJ2000
[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0. -0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0. -0.  1.]]

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -0.  1.]]


Indeed, **they are identity matrices** as expected, so ``ECLIPJ2000_DE405`` == ``ECLIPJ2000``.

## Ceres State Vector and Orbital Elements

Now, for practice,
1. Get state vector of Ceres
2. Convert it to orbital elements
    * [``oscelt``](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/oscelt_c.html): orbital elements etc (8 values) 
    * [``oscltx``](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/oscltx_c.html): ``oscelt`` + 3 more (incl. orb. period) 
```
        [0] RP      Perifocal distance. [km]
        [1] ECC     Eccentricity.
        [2] INC     Inclination.        [rad]
        [3] LNODE   Longitude of the ascending node.  [rad]
        [4] ARGP    Argument of periapsis.  [rad]
        [5] M0      Mean anomaly at epoch.  [rad]
        [6] T0      Epoch.
        [7] MU      Gravitational parameter.
        [8] NU      True anomaly at epoch.  [rad]
        [9] A       Semi-major axis. 0 if it is not computable.  [km]
        [10] TAU    Orbital period. 0 if not elliptical orbit.   [sec]
```
3. Compare with SBDB value
4. Convert the orbital elements to the state vector and compare.
    * [``conics``](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/conics_c.html): First 8 elems (``[:8]``) from above and ET as input.

In [28]:
ceres_sol, _lt = sp.spkgeo(targ=2000001, et=NOW_ET, ref='ECLIPJ2000', obs=10)
_ceres_elt = sp.oscltx(state=ceres_sol, et=NOW_ET, mu=GM_SUN)
ceres_elt = dict(zip(['q', 'e', 'i', 'om', 'w', 'm0', 't0', "mu", "nu", "a", "per"], _ceres_elt))
ceres_elt['q'] = sp.convrt(ceres_elt['q'], 'km', 'au')
ceres_elt['a'] = sp.convrt(ceres_elt['a'], 'km', 'au')
ceres_elt['i'] = sp.convrt(ceres_elt['i'], 'radians', 'degrees')
ceres_elt['om'] = sp.convrt(ceres_elt['om'], 'radians', 'degrees')
ceres_elt['w'] = sp.convrt(ceres_elt['w'], 'radians', 'degrees')
ceres_elt['per'] = sp.convrt(ceres_elt['per'], 'seconds', 'days')

ceres_sbdb = SBDB.query('Ceres')
ceres_hori = QTable(Horizons(id='1', location='500@10', epochs=NOW_UTC.jd, id_type='smallbody').elements())
ceres_hori.rename_columns(["incl", "Omega"], ["i", "om"])

print("elem SPICE      HORIZONS    SBDB")
for key, value in ceres_elt.items():
    try:
        print(f"{key:<4s}", f"{value:<10.3f}", f"{ceres_hori[key][0].value:<10.3f}", f"{ceres_sbdb['orbit']['elements'][key]:<10.3f}")
    except KeyError:
        continue
print("epoch", ceres_elt['t0'],
      "(unavailable from astroquery)",
      sp.utc2et(Time(ceres_sbdb['orbit']['epoch'], format="jd", scale="tdb").utc.iso))

ceres_sol_test = np.array(sp.conics(_ceres_elt[:8], NOW_ET))

np.testing.assert_array_almost_equal(ceres_sol, ceres_sol_test)
print("\n\nTask #4: re-calculated state vector match the one from SPICE")

elem SPICE      HORIZONS    SBDB
q    2.549      2.549      2.550      AU
e    0.079      0.079      0.079     
i    10.587     10.587     10.600     deg
om   80.254     80.254     80.300     deg
w    73.397     73.397     73.400     deg
a    2.767      2.767      2.770      AU
epoch 753991687.1828909 (unavailable from astroquery) 747835200.0004691


Task #4: re-calculated state vector match the one from SPICE


## Close Fly-by Object

On spaceweather.com we can see that an asteroid has a close Earth fly-by:

> 136795(1997BQ) on 2020-05-21 at a distance of 16.1 Lunar Distance

1. What is the distance to the Earth at NOW_ET?
2. Compare it with HORIZONS query

In [193]:
SBDB.query('136795')["orbit"]["epoch"]

<Quantity 2460200.5 d>

In [31]:
# On spaceweather.com we can see that an asteroid has a close Earth fly-by:
# 136795(1997BQ) on 2020-May-21 at a distance of 16.1 Lunar Distance

neo_elt_sbdb_epoch = SBDB.query('136795')['orbit']['epoch']
neo_elt_sbdb = SBDB.query('136795')['orbit']['elements']
neo_sol = np.array(sp.conics([
    neo_elt_sbdb['q'].to_value("km"),
    neo_elt_sbdb['e'],
    neo_elt_sbdb['i'].to_value("rad"),
    neo_elt_sbdb['om'].to_value("rad"),
    neo_elt_sbdb['w'].to_value("rad"),
    neo_elt_sbdb['ma'].to_value("rad"),
    # sp.str2et(Time(neo_elt_sbdb['tp'], format="jd", scale="tdb").strftime("%Y-%m-%d %H:%M:%S") + " TDB"),  # already in TDB == ET
    sp.utc2et(Time(neo_elt_sbdb_epoch, format="jd", scale="tdb").utc.iso),  # already in TDB == ET
    GM_SUN],
    NOW_ET
))
earth_sol, _lt = sp.spkgeo(399, et=NOW_ET, ref='ECLIPJ2000', obs=10)

neo_horizons = Horizons(id='136795', epochs=[NOW_UTC.jd], id_type='smallbody').ephemerides()


# Set one Lunar Distance (LD) in km (value from spaceweather.com)
dist_ld = sp.vnorm(earth_sol[:3] - neo_sol[:3]) / 384401.0
print("From SPICE    [LD] : ", dist_ld)
print("From HORIZONS [LD] : ", neo_horizons['delta'].to("km").value[0] / 384401.0)

From SPICE    [LD] :  711.0185453010555
From HORIZONS [LD] :  711.7682969046051
